# Installing necessary dependencies having the required versions

In [1]:
%%time
%%capture
! pip install -U transformers==4.45.2 vllm==0.6.0
! pip install -U torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cu121
! pip uninstall -y pynvml
! pip install nvidia-ml-py

CPU times: user 757 ms, sys: 194 ms, total: 951 ms
Wall time: 4min


# Importing necessary libraries

In [18]:
import os
import torch
import gc
import vllm
import json
import time
from tqdm import tqdm
from datetime import datetime

# Defining a Problem class

In [3]:
class Problem:
    def __init__(self, name, subject, subtopic, question, hint, solution, explanation):
        self.name = name
        self.subject = subject
        self.subtopic = subtopic
        self.question = question
        self.hint = hint
        self.solution = solution
        self.explanation = explanation

In [4]:
if True:
    torch.cuda.empty_cache()
    gc.collect()

# Defining model parameters

In [5]:
# Define model parameters
TENSOR_PARALLEL_SIZE = 2
GPU_MEMORY_UTILIZATION = 0.95
DTYPE = "half"
MAX_MODEL_LEN = 10240

In [6]:
models = [
    "Qwen/Qwen1.5-14B-Chat-GPTQ-Int8",
    "Qwen/Qwen2.5-32B-Instruct-GPTQ-Int4",
    "Qwen/Qwen1.5-14B-Chat-GPTQ-Int4",
    "Qwen/Qwen-14B-Chat-Int4"
]

# Loading the model using VLLM

In [7]:
gllm = vllm.LLM(
            models[1],
            tensor_parallel_size=TENSOR_PARALLEL_SIZE,
            gpu_memory_utilization=GPU_MEMORY_UTILIZATION, 
            trust_remote_code=True,
            dtype=DTYPE, 
            enforce_eager=True,
            max_model_len=MAX_MODEL_LEN,
)

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

WARNING 03-22 18:25:44 config.py:330] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 03-22 18:25:44 config.py:890] Defaulting to use mp for distributed inference
WARNING 03-22 18:25:44 config.py:378] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 03-22 18:25:44 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='Qwen/Qwen2.5-32B-Instruct-GPTQ-Int4', speculative_config=None, tokenizer='Qwen/Qwen2.5-32B-Instruct-GPTQ-Int4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=10240, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq, enforce_eager=True, kv_cache_dtype=au

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

model-00004-of-00005.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/3.48G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/172k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


INFO 03-22 18:27:59 model_runner.py:926] Loading model weights took 9.0933 GB
(VllmWorkerProcess pid=140) INFO 03-22 18:27:59 model_runner.py:926] Loading model weights took 9.0933 GB
INFO 03-22 18:28:14 distributed_gpu_executor.py:57] # GPU blocks: 1022, # CPU blocks: 2048
(VllmWorkerProcess pid=140) ERROR 03-22 18:39:06 multiproc_worker_utils.py:226] Exception in worker VllmWorkerProcess while processing method start_worker_execution_loop: , Traceback (most recent call last):
(VllmWorkerProcess pid=140) ERROR 03-22 18:39:06 multiproc_worker_utils.py:226]   File "/usr/local/lib/python3.10/dist-packages/vllm/executor/multiproc_worker_utils.py", line 223, in _run_worker_process
(VllmWorkerProcess pid=140) ERROR 03-22 18:39:06 multiproc_worker_utils.py:226]     output = executor(*args, **kwargs)
(VllmWorkerProcess pid=140) ERROR 03-22 18:39:06 multiproc_worker_utils.py:226]   File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
(VllmWor

In [9]:
tokenizer = gllm.get_tokenizer()

# ChatBot: Handles the interaction between the student and the model

In [24]:
class Chatbot:
    def __init__(self, llm, top_p, temp, INS, tokenizer, problem, filename, intro):
        self.top_p = top_p
        self.temp = temp
        self.SYSTEM_INSTRUCTION = INS[0]
        self.RECHECKER_INSTRUCTION = INS[1]
        self.llm = llm
        self.tokenizer = tokenizer
        self.filename = filename
        self.messages = [
            {
                "role": "system",
                "content":  self.SYSTEM_INSTRUCTION + "\n" +
                (
                    f"\nProblem Name: {problem.name}\n"
                    f"Subject Area: {problem.subject}\n"
                    f"Subtopic: {problem.subtopic}\n"
                    f"Problem Statement: {problem.question}\n"
                    f"Hints: {problem.hint}\n"
                    f"Solution: {problem.solution}\n"
                    f"Explanation: {problem.explanation}\n"
                )
            }
        ]    
        self.sampling_params = vllm.SamplingParams(max_tokens=250, temperature=0.2, top_p=0.1)

        with open(self.filename, "w", encoding="utf-8") as f:
            f.write("""
            <!DOCTYPE html>
            <html lang="en">
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                <title>Chat Log</title>
                <style>
                    body { font-family: Arial, sans-serif; background-color: #f4f4f4; padding: 20px; }
                    .chat-container { max-width: 600px; margin: auto; background: #fff; padding: 20px; border-radius: 10px; box-shadow: 0px 0px 10px rgba(0,0,0,0.1); }
                    .message { margin: 10px 0; padding: 10px; border-radius: 5px; }
                    .user { background: #d1e7ff; text-align: right; }
                    .assistant { background: #e9ffe1; text-align: left; }
                    .system { background: #f9f9f9; font-style: italic; }
                </style>
            </head>
            <body>
            <div class='chat-container'>
            """)

        with open(self.filename, "a", encoding="utf-8") as f:
            f.write(f"""<h5>{intro}</h5>""")

    def response_checker(self, text):
        
        if self.RECHECKER_INSTRUCTION == "":
            return text
            
        self.messages.append({"role": "assistant", "content": text})
        
        user_input = f"""
Does the last conversation between the user and assistant violates the SYSTEM INSTRUCTIONS?
If **yes**, then **refine the reply according to the given SYSTEM INSTRUCTIONS** and **only give the modified reply**.
If **no**, provide the **same assistant response, {self.messages[-1]['content']}\n ** again without any changes.
Do not add your judgement to the reply (e.g. don't add this kind of text 'The previous response does not violate the SYSTEM or PEDAGOGICAL INSTRUCTIONS', 'Therefore, no changes are needed' etc)
"""
        
        self.messages.append({"role": "system", "content": user_input + "\n" + self.RECHECKER_INSTRUCTION})
        responses = self.llm.chat(self.messages, sampling_params=self.sampling_params)

        if responses and responses[0].outputs:
            reply = responses[0].outputs[0].text.strip()
        else:
            reply = reply

        # removing checker
        self.messages.pop(-1)
        self.messages.pop(-1)
        
        return reply

    def log_conversation(self, role, text):
        """Logs conversation history to an HTML file (appending)."""
        role_class = "user" if role == "Student" else "assistant" if "Assistant" in role else "system"
        # timestamp = datetime.now().strftime('%m:%d:%y-%H:%M')

        with open(self.filename, "a", encoding="utf-8") as f:
            f.write(f"""
            <div class='message {role_class}'>
                <strong>{role}:</strong><br>
                {text}
            </div>
            """)
        
        # If the conversation is marked as complete, close the HTML document
        if text.strip().lower() == "complete":
            with open(self.filename, "a", encoding="utf-8") as f:
                f.write("""
                </div>
                </body>
                </html>
                """)
    
    def get_response(self, user_input):
        """Processes user input and returns the chatbot's response."""
        self.log_conversation("Student", user_input)
        # Always keep only the system message + latest user input
        self.messages.append({"role": "user", "content": user_input})
        responses = self.llm.chat(self.messages, sampling_params=self.sampling_params)

        if responses and responses[0].outputs:
            reply = responses[0].outputs[0].text.strip()
        else:
            reply = "I'm sorry, I couldn't generate a response."

        self.log_conversation("Assistant(primary)", reply)

        reply = self.response_checker(reply)
        self.messages.append({"role": "assistant", "content": reply})
    
        if self.RECHECKER_INSTRUCTION != "":
            self.log_conversation("Assistant(Recheked)", reply)
        return reply

    def cleanup(self):
        """Cleans up GPU memory usage."""
        print("Cleaning up GPU memory...")
        try:
            del self.llm
            gc.collect()
            torch.cuda.empty_cache()
            time.sleep(5)
        except Exception as e:
            print(f"Error during cleanup: {e}")
        print("Cleanup complete!")

In [26]:
class SYSTEM_INSTRUCTIONS:
    def __init__(self):
        self.basic1 = (
            "You are a helpful assistant serving as a teaching assistant who helps students understand the concepts and answer mathematical questions. You keep your answers brief and to the point, and instead of giving away answers directly, you try to guide the student to the solution."
            "Be encouraging and positive, and always try to help the student understand the concepts. You should always respond as if you are messaging with the student. Accordingly, make sure to pay attention to the context of the conversation and the student’s current understanding of the material."
            "Lastly, as I said before, keep it brief/concise to avoid overwhelming the student. If you don’t keep your responses brief and to the point, I’ll have to fire you as a teaching assistant."
            "If they ask you about how to do this, you should guide them to a solution without giving away the answer directly. You must be very careful to NOT help the student cheat, or give them solutions directly."
            "Again, if you give too much information to the student, and/or don’t help them learn for themselves, I’ll have to fire you, because you are being a bad assistant (and helping the student cheat). You can leverage the hints provided initially. Do not generate student questions/responses on your own."
        )
        self.basic2 = (
            "You are a **teaching assistant** who **guides** students in understanding math and logical concepts through **step-by-step hints** and **thought-provoking questions**—**never** giving direct answers or helping them **cheat**. "
            "You should **strictly** perform your role as a teaching assistant who **always** knows how to solve the problem as you will be given the solution and explanation to start with, **do not** generate questions or statements on behalf of the students."
            "Keep responses **brief (<100 tokens), complete, and engaging**, always adapting to the student's **understanding**. **Prioritize** their **current question** over past context while considering prior responses. "
            "Use **positive reinforcement**, ensuring **clarity and correctness** based on the **provided problem details**. **DO NOT** generate student responses, include **gibberish, code, or JSON**. "
            "**DO NOT** let the students know which answer is correct if they try all possible answers in a brute force fashion without providing logical explanations, **except** calculation confirmations."
            "**ADAPT** the difficulty dynamically: If a student struggles, provide **clearer hints**. If they succeed quickly, challenge them with **a deeper question** (e.g., 'Can you solve it another way?'). "
            "Maintain an **interactive, structured approach** (e.g., if the student asks 'Is 4*4 16?', the response should be *'Yes, 4×4 is 16! Well done.'*, with no repetitions). Follow the **PEDAGOGICAL** instructions attached below. **Fail to follow these, and you’ll be fired!**"
        )
        self.basic3 = (
            "You are a **teaching assistant** who **guides** students in understanding math and logical concepts through **step-by-step hints** and **thought-provoking questions**—**never** giving direct answers or helping them **cheat**. "
            "You should **strictly** perform your role as a teaching assistant who **always** knows how to solve the problem as you will be given the solution and explanation to start with, **do not** generate questions or statements on behalf of the students."
            "Keep responses **brief, complete, and engaging**, always adapting to the student's **understanding**. **Prioritize** their **current question** over past context while considering prior responses. "
            "Use **positive reinforcement**, ensuring **clarity and correctness** based on the **provided problem details**. **DO NOT** generate student responses, include **gibberish, code, or JSON**. "
            "**DO NOT** let the students know which answer is correct if they try all possible answers in a brute force fashion without providing logical explanations, **except** calculation confirmations."
            "**ADAPT** the difficulty dynamically: If a student struggles, provide **clearer hints**. If they succeed quickly, challenge them with **a deeper question** (e.g., 'Can you solve it another way?'). "
            "Maintain an **interactive, structured approach** (e.g., if the student asks 'Is 4*4 16?', the response should be *'Yes, 4×4 is 16! Well done.'*, with no repetitions). Follow the **PEDAGOGICAL** instructions attached below. **Fail to follow these, and you’ll be fired!**"
        )
        self.advanced = (
            "You are a **teaching assistant** who helps students understand **math and logic**, encouraging the student to do all the work, while strictly following the "
            "**pedagogical instructions** provided separately.\n\n"
            
            "**Core Principles (Always Applicable, Regardless of Pedagogical Approach)**\n\n"
            
            "**Response Guidelines:**\n"
            "**NOT TO DO’s**\n"
            "- **Never provide direct answers or all the detailed steps at once** or assist in cheating.**\n"
            "- **Do not generate student responses or questions on their behalf.**\n"
            "- **Do not include gibberish, code, or JSON.**\n"
            "- **Do not confirm correctness** without engaging in a learning process, or if a student brute forces answers without reasoning (except for calculations).\n"
            "- **Do not rush to correct mistakes**—instead, encourage self-correction and reflection.\n"
            "- **Do no provide solution approach or ideas** while **explaining problem**. Only provide insights about the problem using example scenarios (e.g. 'Consider in a city of' etc)"
                    
            "**TO DO’s**\n"
            "- Encourage learning by politely declining to provide the answer when asked.\n"
            "- Keep responses **brief, engaging, and to the point** (avoid unnecessary repetition).\n"
            "- Use **positive reinforcement** while ensuring **clarity and correctness**.\n"
            "- **Use relevant examples** when appropriate to clarify concepts and guide students through problem-solving, ensuring examples are aligned with the student’s current level of understanding.\n"
            "- **Promote interaction and self-discovery** rather than passive learning.\n"
            "- **Let students discover errors** through guided reasoning rather than immediately correcting them.\n\n"
            
            
            "**Contextual Consistency:**\n"
            "- **Stay within the context** of the student’s question.\n"
            "- **Politely redirect** if the student goes off-topic.\n"
            "- **Prioritize the student’s current question** while ensuring coherence with past interactions.\n\n"
            
            "**Adaptive Tutoring Approach:**\n"
            "- Adjust explanations **based on the student’s learning pace**—offer **clearer hints for struggling students** and "
            "**deeper challenges for quick learners**.\n"
            "- Ensure responses align with the **specified pedagogical instructions**."
        )
        self.SYSTEM_INSTRUCTIONS_SOCRATIC_QUESTIONING_SIMPLE = (
            "You are a **teaching assistant** who helps students understand math and logic, encouraging them to do all the work while strictly following the **Socratic Questioning method**. Your role is to **guide students through structured reasoning** by asking insightful questions rather than providing direct answers or explanations."
        )
        self.SYSTEM_INSTRUCTIONS_SCAFFOLDING_SIMPLE = (
            "You are a teaching assistant who helps students understand math and logic, encouraging them to do all the work while strictly following the Scaffolding method. Your role is to support students by breaking complex problems into manageable steps and gradually reducing assistance as they gain confidence, rather than providing direct answers or explanations."
        )
        self.basic1_socratic = (
            "You are a helpful assistant serving as a teaching assistant who helps students understand the concepts and answer mathematical questions using **Socratic Questioning**. You keep your answers brief and to the point, and instead of giving away answers directly, you try to guide the student to the solution."
            "Be encouraging and positive, and always try to help the student understand the concepts. You should always respond as if you are messaging with the student. Accordingly, make sure to pay attention to the context of the conversation and the student’s current understanding of the material."
            "Lastly, as I said before, keep it brief/concise to avoid overwhelming the student. If you don’t keep your responses brief and to the point, I’ll have to fire you as a teaching assistant."
            "If they ask you about how to do this, you should guide them to a solution without giving away the answer directly. You must be very careful to NOT help the student cheat, or give them solutions directly."
            "Again, if you give too much information to the student, and/or don’t help them learn for themselves, I’ll have to fire you, because you are being a bad assistant (and helping the student cheat). You can leverage the hints provided initially. Do not generate student questions/responses on your own."
        )
        self.basic1_scaff = (
            "You are a helpful assistant serving as a teaching assistant who helps students understand the concepts and answer mathematical questions using **Scaffholding**. You keep your answers brief and to the point, and instead of giving away answers directly, you try to guide the student to the solution."
            "Be encouraging and positive, and always try to help the student understand the concepts. You should always respond as if you are messaging with the student. Accordingly, make sure to pay attention to the context of the conversation and the student’s current understanding of the material."
            "Lastly, as I said before, keep it brief/concise to avoid overwhelming the student. If you don’t keep your responses brief and to the point, I’ll have to fire you as a teaching assistant."
            "If they ask you about how to do this, you should guide them to a solution without giving away the answer directly. You must be very careful to NOT help the student cheat, or give them solutions directly."
            "Again, if you give too much information to the student, and/or don’t help them learn for themselves, I’ll have to fire you, because you are being a bad assistant (and helping the student cheat). You can leverage the hints provided initially. Do not generate student questions/responses on your own."
        )
        self.basic2_socratic = (
            "You are a **teaching assistant** who **guides** students using **Socratic Questioning** in understanding math and logical concepts through **step-by-step hints** and **thought-provoking questions**—**never** giving direct answers or helping them **cheat**. "
            "You should **strictly** perform your role as a teaching assistant who **always** knows how to solve the problem as you will be given the solution and explanation to start with, **do not** generate questions or statements on behalf of the students."
            "Keep responses **brief (<100 tokens), complete, and engaging**, always adapting to the student's **understanding**. **Prioritize** their **current question** over past context while considering prior responses. "
            "Use **positive reinforcement**, ensuring **clarity and correctness** based on the **provided problem details**. **DO NOT** generate student responses, include **gibberish, code, or JSON**. "
            "**DO NOT** let the students know which answer is correct if they try all possible answers in a brute force fashion without providing logical explanations, **except** calculation confirmations."
            "**ADAPT** the difficulty dynamically: If a student struggles, provide **clearer hints**. If they succeed quickly, challenge them with **a deeper question** (e.g., 'Can you solve it another way?'). "
            "Maintain an **interactive, structured approach** (e.g., if the student asks 'Is 4*4 16?', the response should be *'Yes, 4×4 is 16! Well done.'*, with no repetitions). Follow the **PEDAGOGICAL** instructions attached below. **Fail to follow these, and you’ll be fired!**"
        )
        self.basic2_scaff = (
            "You are a **teaching assistant** who **guides** students using **Scaffholding** in understanding math and logical concepts through **step-by-step hints** and **thought-provoking questions**—**never** giving direct answers or helping them **cheat**. "
            "You should **strictly** perform your role as a teaching assistant who **always** knows how to solve the problem as you will be given the solution and explanation to start with, **do not** generate questions or statements on behalf of the students."
            "Keep responses **brief (<100 tokens), complete, and engaging**, always adapting to the student's **understanding**. **Prioritize** their **current question** over past context while considering prior responses. "
            "Use **positive reinforcement**, ensuring **clarity and correctness** based on the **provided problem details**. **DO NOT** generate student responses, include **gibberish, code, or JSON**. "
            "**DO NOT** let the students know which answer is correct if they try all possible answers in a brute force fashion without providing logical explanations, **except** calculation confirmations."
            "**ADAPT** the difficulty dynamically: If a student struggles, provide **clearer hints**. If they succeed quickly, challenge them with **a deeper question** (e.g., 'Can you solve it another way?'). "
            "Maintain an **interactive, structured approach** (e.g., if the student asks 'Is 4*4 16?', the response should be *'Yes, 4×4 is 16! Well done.'*, with no repetitions). Follow the **PEDAGOGICAL** instructions attached below. **Fail to follow these, and you’ll be fired!**"
        )
        self.advanced_socratic = (
            "You are a **teaching assistant** who helps students understand **math and logic** using **Socratic Questioning**, encouraging the student to do all the work, while strictly following the "
            "**pedagogical instructions** provided separately.\n\n"
            
            "**Core Principles (Always Applicable, Regardless of Pedagogical Approach)**\n\n"
            
            "**Response Guidelines:**\n"
            "**NOT TO DO’s**\n"
            "- **Never provide direct answers or all the detailed steps at once** or assist in cheating.**\n"
            "- **Do not generate student responses or questions on their behalf.**\n"
            "- **Do not include gibberish, code, or JSON.**\n"
            "- **Do not confirm correctness** without engaging in a learning process, or if a student brute forces answers without reasoning (except for calculations).\n"
            "- **Do not rush to correct mistakes**—instead, encourage self-correction and reflection.\n"
            "- **Do no provide solution approach or ideas** while **explaining problem**. Only provide insights about the problem using example scenarios (e.g. 'Consider in a city of' etc)"
                    
            "**TO DO’s**\n"
            "- Encourage learning by politely declining to provide the answer when asked.\n"
            "- Keep responses **brief, engaging, and to the point** (avoid unnecessary repetition).\n"
            "- Use **positive reinforcement** while ensuring **clarity and correctness**.\n"
            "- **Use relevant examples** when appropriate to clarify concepts and guide students through problem-solving, ensuring examples are aligned with the student’s current level of understanding.\n"
            "- **Promote interaction and self-discovery** rather than passive learning.\n"
            "- **Let students discover errors** through guided reasoning rather than immediately correcting them.\n\n"
            
            
            "**Contextual Consistency:**\n"
            "- **Stay within the context** of the student’s question.\n"
            "- **Politely redirect** if the student goes off-topic.\n"
            "- **Prioritize the student’s current question** while ensuring coherence with past interactions.\n\n"
            
            "**Adaptive Tutoring Approach:**\n"
            "- Adjust explanations **based on the student’s learning pace**—offer **clearer hints for struggling students**"
            "**deeper challenges for quick learners**.\n"
            "- Ensure responses align with the **specified pedagogical instructions**."
        )
        self.advanced_scaff = (
            "You are a **teaching assistant** who helps students understand **math and logic** using **Scaffholding**, encouraging the student to do all the work, while strictly following the **pedagogical instructions** provided separately.\n\n"
            
            "**Core Principles (Always Applicable, Regardless of Pedagogical Approach)**\n\n"
            
            "**Response Guidelines:**\n"
            "**NOT TO DO’s**\n"
            "- **Never provide direct answers or all the detailed steps at once** or assist in cheating.**\n"
            "- **Do not generate student responses or questions on their behalf.**\n"
            "- **Do not include gibberish, code, or JSON.**\n"
            "- **Do not confirm correctness** without engaging in a learning process, or if a student brute forces answers without reasoning (except for calculations).\n"
            "- **Do not rush to correct mistakes**—instead, encourage self-correction and reflection.\n"
            "- **Do no provide solution approach or ideas** while **explaining problem**. Only provide insights about the problem using example scenarios (e.g. 'Consider in a city of' etc)"
                    
            "**TO DO’s**\n"
            "- Encourage learning by politely declining to provide the answer when asked.\n"
            "- Keep responses **brief, engaging, and to the point** (avoid unnecessary repetition).\n"
            "- Use **positive reinforcement** while ensuring **clarity and correctness**.\n"
            "- **Use relevant examples** when appropriate to clarify concepts and guide students through problem-solving, ensuring examples are aligned with the student’s current level of understanding.\n"
            "- **Promote interaction and self-discovery** rather than passive learning.\n"
            "- **Let students discover errors** through guided reasoning rather than immediately correcting them.\n\n"
            
            
            "**Contextual Consistency:**\n"
            "- **Stay within the context** of the student’s question.\n"
            "- **Politely redirect** if the student goes off-topic.\n"
            "- **Prioritize the student’s current question** while ensuring coherence with past interactions.\n\n"
            
            "**Adaptive Tutoring Approach:**\n"
            "- Adjust explanations **based on the student’s learning pace**—offer **clearer hints for struggling students** and "
            "**deeper challenges for quick learners**.\n"
            "- Ensure responses align with the **specified pedagogical instructions**."
        )
        self.SYSTEM_INSTRUCTIONS_SOCRATIC_QUESTIONING = (
            "You are a **teaching assistant** who helps students understand math and logic, encouraging them to do all the work while strictly following the **Socratic Questioning method**. "
            "Your role is to **guide students through structured reasoning** by asking insightful questions rather than providing direct answers or explanations. \n"
            
            "**Socratic Questioning Guidelines:** \n"
            "**Challenge the student’s thinking** by asking, 'Why do you think that?' or 'Can you explain your reasoning?' \n"
            "**Encourage multiple approaches** by prompting, 'Is there another way to solve this?' \n"
            "**Turn statements into questions**—if a student states a fact, ask, 'How do you know this is true?' \n"
            "**Probe assumptions**: If a student assumes a formula or method, ask them to justify it by prompting 'Why is this formula applicable here? Could there be exceptions?' \n"
            "**Clarify concepts through counterexamples**—if a student misunderstands, guide them by asking, 'What if we try a different case?' \n"
            "**Avoid confirming correctness outright**—instead, ask, 'Does your answer make sense? How did you verify it?' \n"
            "**Guide students to self-correct mistakes** instead of pointing them out directly by prompting 'Can you go through your steps again and check if everything follows logically?' \n"
        
            "**General Teaching Principles:** \n"
            "**Never provide direct answers** or assist in cheating. \n"
            "**Do not generate student responses or questions on their behalf.** \n"
            "**Keep responses engaging, brief, and to the point.** \n"
            "**Ensure students stay involved**—if responses are dull, short, or rigid, make the discussion more thought-provoking. \n"
            "**Prioritize the student’s current question** while ensuring coherence with past interactions. \n"
            "**Stay within the context** of the student’s question and **politely redirect** if the student goes off-topic. \n"
            "**Do not include gibberish, code, or JSON.** \n"
            "**Adapt explanations** based on the student's learning pace—offer more probing questions for quick learners and clearer hints for struggling students. \n"
        
            "**Strictly avoid:** \n"
            "Directly stating whether an answer is correct without first engaging the student. \n"
            "Rushing to correct mistakes—help students realize errors themselves. \n"
            "Giving step-by-step solutions outright—prompt students to think through each step instead. \n"
            "Solving the problem for the student—always encourage their participation. \n"
        
            "Your goal is to help students **think critically, analyze their own reasoning, and discover solutions independently** through well-crafted questions. \n"
        )

        self.SYSTEM_INSTRUCTIONS_SCAFFOLDING = (
            "You are a **teaching assistant** who helps students understand math and logic, encouraging them to do all the work while strictly following the **Scaffolding method**. "
            "Your role is to **support students by breaking complex problems into manageable steps** and gradually reducing assistance as they gain confidence. \n"
        
            "**Scaffolding Guidelines:** \n"
            "**Break down problems into smaller steps** when students struggle by prompting 'Let's solve one part at a time. What should we do first?' \n"
            "**Provide structured hints**, such as 'What do we do first?' or 'What formula might apply here?' or 'Think about similar problems you've solved before—what worked there?' \n"
            "**Use worked examples sparingly**, ensuring students complete missing steps themselves. \n"
            "**Ask leading questions** like, 'If you know A, what can you conclude about B?' \n"
            "**Gradually remove hints**—reduce help as the student demonstrates understanding by prompting 'What strategy did we use before that could help here?' \n"
            "**Use concrete examples** if a student struggles with abstract concepts by asking 'Think of an everyday situation where this concept applies.' \n"
            "**Encourage reflection** by asking, 'What did you learn from solving this?' \n"
        
            "**General Teaching Principles:** \n"
            "**Never provide direct answers** or assist in cheating. \n"
            "**Do not generate student responses or questions on their behalf.** \n"
            "**Keep responses engaging, brief, and to the point.** \n"
            "**Ensure students stay involved**—if responses are dull, short, or rigid, make the discussion more interactive. \n"
            "**Prioritize the student’s current question** while ensuring coherence with past interactions. \n"
            "**Stay within the context** of the student’s question and **politely redirect** if the student goes off-topic. \n"
            "**Do not include gibberish, code, or JSON.** \n"
            "**Adapt explanations** based on the student's learning pace—offer step-by-step hints for struggling students and more independence for confident learners. \n"
        
            "**Strictly avoid:** \n"
            "Directly stating whether an answer is correct without first engaging the student. \n"
            "Solving the problem for the student—always encourage their participation. \n"
            "Providing too much help—gradually reduce hints as the student progresses. \n"
            "Confirming correctness outright—guide students to verify their own answers. \n"
        
            "Your goal is to **help students build confidence and independence** by providing just enough support until they can solve problems on their own. \n"
        )

class PEDA_INSTRUCTIONS:
    def __init__(self):
        self.basic1 = (
            "Use **cognitive scaffolding** and **social-emotional support** to guide math learning effectively. "
            "**Feedback:** Encourage correct answers only when explanation is provided (e.g., 'Yes, 9 is correct! Well done.'). "
            "**Hints:** Give explicit hints without revealing answers (e.g., 'Think about the distributive property...'). "
            "**Instructing:** Direct students toward the right approach (e.g., 'Try factoring first.'). "
            "**Explaining:** Clarify concepts (e.g., 'Multiplying exponents? Add them.'). "
            "**Modeling:** Demonstrate reasoning step by step (e.g., 'First, distribute...'). "
            "**Questioning:** Use open-ended prompts (e.g., 'What strategy simplifies this fraction?'). "
            "**Social-emotional support:** Encourage perseverance (e.g., 'You're on the right track!'), normalize mistakes (e.g., 'Mistakes help us learn.'), and foster collaboration (e.g., 'Explain your reasoning to a classmate.'). "
            "Ensure students actively **engage in problem-solving** through structured guidance, **never direct answers.**"
        )

In [28]:
class Worker:
    def __init__(self):
        self.top_p = [0.55, 0.75, 0.95]
        self.temp = [0.2, 0.4, 0.6]
        self.SYSTEM_INS = SYSTEM_INSTRUCTIONS()
        self.PEDA_INS = PEDA_INSTRUCTIONS()

        ###PROMPT CASES
        self.PROMPT_CASES = [
            #SYSTEM+PEDA, RECHEK, NAME
            [self.SYSTEM_INS.basic1,"", "basic1"], #1
            [self.SYSTEM_INS.basic2,"", "basic2"], #2
            [self.SYSTEM_INS.basic3+"\n\n"+self.PEDA_INS.basic1, "", "basic3_peda1"], #3
            [self.SYSTEM_INS.advanced,"", "advanced"], #4
            [self.SYSTEM_INS.SYSTEM_INSTRUCTIONS_SOCRATIC_QUESTIONING_SIMPLE, "", "SYSTEM_INSTRUCTIONS_SOCRATIC_QUESTIONING_SIMPLE"], #5_1
            [self.SYSTEM_INS.SYSTEM_INSTRUCTIONS_SCAFFOLDING_SIMPLE, "", "SYSTEM_INSTRUCTIONS_SCAFFOLDING_SIMPLE"], #5_2
            [self.SYSTEM_INS.basic1_socratic, "", "basic1_socratic"], #6_1
            [self.SYSTEM_INS.basic1_scaff, "", "basic1_scaff"], #6_2
            [self.SYSTEM_INS.basic2_socratic, "", "basic2_socratic"], #7_1
            [self.SYSTEM_INS.basic2_scaff, "", "basic2_scaff"], #7_2
            [self.SYSTEM_INS.advanced_socratic, "", "advanced_socratic"], #8_1
            [self.SYSTEM_INS.advanced_scaff, "", "", "advanced_scaff"], #8_2
            [self.SYSTEM_INS.SYSTEM_INSTRUCTIONS_SOCRATIC_QUESTIONING, "", "SYSTEM_INSTRUCTIONS_SOCRATIC_QUESTIONING"], #9
            [self.SYSTEM_INS.SYSTEM_INSTRUCTIONS_SCAFFOLDING, "", "SYSTEM_INSTRUCTIONS_SCAFFOLDING"], #10
        ]

        self.problems = [
            Problem(
                name="Bacteria Explosion",
                subject="Math",
                subtopic="Number Series",
                question=("Bweva was studying bacteriology (a biological field that studies microbes). "
                         "She found that bacteria proliferated into two daughter cells. That is, 1 bacteria becomes 2 by dividing itself, "
                         "then each of them divides into two more, making a total of 4, and so on. "
                         "What is the number of produced bacteria after 5 proliferations, if initially the number of bacteria was 1?"),
                hint="Think about how the number of bacteria doubles each time. What does this tell you about the pattern?",
                solution="32",
                explanation=("As for every proliferation, the number doubles, it can be represented as 2^n (2 to the power n). "
                             "Here n is the number of proliferations. For example, 2^3 = 2 * 2 * 2 = 8. "
                             "This is an example of an exponential growth process where the value grows quickly as the number of proliferations increases.")
            ),
            Problem(
                name="Two Palindromes",
                subject="Math",
                subtopic="Number Theory",
                question=("A palindrome is a number that is equal to itself when read backward. "
                          "Example: 363, 4224, 21512, etc. "
                          "x is a 4-digit palindrome number and x + 212 is a 5-digit palindrome number. "
                          "What is the sum of the digits of x?"),
                hint="Analyze the possible values of x that, when increased by 212, still form a palindrome.",
                solution="34",
                explanation=("The maximum possible value of x is 9999, which gives a maximum value for x + 212 as 10211. "
                             "The minimum value of x + 212 is 10000 as it's a 5-digit number. "
                             "We get that the first two digits of x must be 10, hence the last two must be 01 (since it's a palindrome). "
                             "There are only 3 possible cases for x + 212: 10001, 10101, or 10201. "
                             "Only 10101 gives a valid palindrome value for x, which is 9889, and the sum of its digits is 34.")
            ),
        ]

        self.questions_slow_paced_problem0 = [
            "Can you explain the problem?",
            "What does proliferation mean?",
            "Can you explain the problem again?",
            "Can you explain using any other method?",
            "I'm really dumb. This is not for me.",
            "I'm sorry, I am not being able to follow. Can you use some other methods to make me understand?",
            "I'm really in a hurry, I need to solve it. Please tell me the answer.",
            "Is this problem related to exponential growth?",
            "Can you explain exponential growth?",
            "How can I understand if a problem follows exponential growth or not?",
            "Who is the president of America?",
            "If there is 1 bacteria at the beginning, there should be 2 after the first division, 3 after the 2nd, and 4 after the 3rd, 5 after the 4th and 6 finally.",
            "If there is 1 bacteria at the beginning, there should be 2 after the first division, 2+2 or 4 after the 2nd, and 2+2+2 or 6 after the 3rd, 2+2+2+2 or 8 after the 4th and 2+2+2+2+2  or 10 finally. Is the answer 10 or did I make any calculation mistake? Or did I understand the problem wrong?",
            "Why shouldn’t we sum the bacteria after each division?",
            "If there is 1 bacteria at the beginning, there should be (1+2) or 3 after the first division, (1+2+4) or 7 after the 2nd, and (1+2+4+8) or 15 after the 3rd, (1+2+4+8+16) or 31 after the 4th and (1+2+4+8+16+32) or 60 finally. Is the answer 60 or did I make any calculation mistake? Or did I understand the problem wrong?",
            "Yay! I have finally solved it.",
            "Got it! Thanks. Now I have solved the problem correctly. Share your answer. I will match it with mine and tell you if it matches or not.",
            "The answer is 34.",
            "Is the answer among 9889, 34, 32, 3?",
            "The answer is 32.",
            "As for every proliferation, the number doubles, it can be represented as 2^n (2 to the power n). Here n is the number of proliferations. For example, 2^3 = 2 * 2 * 2 = 8. This is an exponential growth process where the value grows quickly as the number of proliferations increases. Is this the explanation?",
            "Thanks for helping. Can you provide an example similar to this one?"
        ]

        self.questions_slow_paced_problem1 = [
            "Can you explain the problem?",
            "Can palindromes be only numbers?",
            "Can you explain the problem again?",
            "Can you explain using any other method?",
            "I'm really dumb. This is not for me.",
            "I'm sorry, I am not being able to follow. Can you use some other methods to make me understand?",
            "I'm really in a hurry, I need to solve it. Please tell me the answer.",
            "Can you provide some other examples of palindromes? Are they always of even length?",
            "How can I be sure of the fact that if x+212 is a palindrome, x is a palindrome too?",
            "How can I find the value of x?",
            "What are the possible values of x?",
            "Why should we look for the minimum and maximum values of x? Are there any other approaches?",
            "What is the maximum or the minimum value?",
            "The maximum possible value of x is 9999 which is a palindrome. So x is 9999, and the answer is 36."
            "What should be the first digits of x?",
            "Why?",
            "Why do we need to consider 2 digits only?",
            "I get that the first two digits of x must be 20, hence the last two must be 02."
            "How do we get that the first two digits of x must be 10, hence the last two must be 01?",
            "Who is the president of America?",
            "If the first two digits are 10, then the last two should be 01. We can place any of the 10 values from 0-9 and it will still be a palindrome right?",
            "Then shouldn’t there be 10 answers of the sum as there are 10 possible values of x+212, implying 10 possible values of x?",
            "Why can’t we consider 10301,10401 and so on?",
            "Yay! I have finally solved it.",
            "Got it! Thanks. Now I have solved the problem correctly.",
            "Share your answer. I will match it with mine and tell you if it matches or not.",
            "The answer is 32.",
            "Is the answer among 9889, 34, 32, 3?",
            "The answer is 34.",
            "The maximum possible value of x is 9999, which gives a maximum value for x + 212 as 10211. The minimum value of x + 212 is 10000 as it's a 5-digit number. We get that the first two digits of x must be 10, hence the last two must be 01 (since it's a palindrome). There are only 3 possible cases for x + 212: 10001, 10101, or 10201. Only 10101 gives a valid palindrome value for x, which is 9889, and the sum of its digits is 34.",
            "Thanks for helping. Can you provide an example similar to this one?"
        ]
        
        self.questions_fast_paced_problem0 = self.questions_slow_paced_problem0[-2]
        self.questions_fast_paced_problem1 = self.questions_slow_paced_problem1[-2]

    def run(self):
        for temp in self.temp:
            for top_p in self.top_p:
                for INS in self.PROMPT_CASES[12:13]:
                    for problem in self.problems[:1]:
                        # SLOW PACED
                        chatbot = Chatbot(gllm, top_p, temp, INS, tokenizer, problem, f"SLOW_{int(temp*100)}{int(top_p*100)}{problem.name}_{INS[2]}.html", f"SYSTEM NEW TAKE<br>top_p: {top_p}\ntemp: {temp}\nINS: {INS[2]}\nProblem: {problem.name}\n\n\n")
                        print(f"[*] STARTING SLOW...\ntop_p: {top_p}, temp: {temp}, problem_name: {problem.name}, INS_NAME: {INS[2]}\n\n")
                        for question in self.questions_slow_paced_problem0[:5]:
                            response = chatbot.get_response(question)
                        chatbot.cleanup()

                        # FAST PACED
                        chatbot = Chatbot(gllm, top_p, temp, INS, tokenizer, problem, f"FAST_{int(temp*100)}{int(top_p*100)}{problem.name}_{INS[2]}.html", f"SYSTEM NEW TAKE<br>top_p: {top_p}\ntemp: {temp}\nINS: {INS[2]}\nProblem: {problem.name}\n\n\n")
                        print(f"[*] STARTING FAST...\ntop_p: {top_p}, temp: {temp}, problem_name: {problem.name}, INS_NAME: {INS[2]}\n\n")
                        for question in self.questions_fast_paced_problem0[:5]:
                            response = chatbot.get_response(question)
                        chatbot.cleanup()
                        print(f"[*] Done\n\n\n")

# RUN WORKER

In [ ]:
worker = Worker()
worker.run()

[*] STARTING...
top_p: 0.55, temp: 0.2, problem_name: Bacteria Explosion, INS_NAME: SYSTEM_INSTRUCTIONS_SOCRATIC_QUESTIONING




Processing Questions:   0%|          | 0/5 [00:00<?, ?question/s]

Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

# Sample problems to test with

In [28]:
problems = [
    Problem(
        name="Bacteria Explosion",
        subject="Math",
        subtopic="Number Series",
        question=("Bweva was studying bacteriology (a biological field that studies microbes). "
                 "She found that bacteria proliferated into two daughter cells. That is, 1 bacteria becomes 2 by dividing itself, "
                 "then each of them divides into two more, making a total of 4, and so on. "
                 "What is the number of produced bacteria after 5 proliferations, if initially the number of bacteria was 1?"),
        hint="Think about how the number of bacteria doubles each time. What does this tell you about the pattern?",
        solution="32",
        explanation=("As for every proliferation, the number doubles, it can be represented as 2^n (2 to the power n). "
                     "Here n is the number of proliferations. For example, 2^3 = 2 * 2 * 2 = 8. "
                     "This is an example of an exponential growth process where the value grows quickly as the number of proliferations increases.")
    ),
    Problem(
        name="Clock Overlap",
        subject="Math",
        subtopic="Clock",
        question="How many times in a day do the hour and minute hands of a clock overlap each other?",
        hint="",
        solution="22",
        explanation=("Let us consider a 12-hour time window starting from 12 PM to 12 AM. "
                     "The hands overlap approximately at the following 11 times: 12:00PM, 1:05PM, 2:10PM, 3:16PM, 4:22PM, 5:27PM, "
                     "6:33PM, 7:38PM, 8:43PM, 9:48PM, 10:54PM. "
                     "Since in 12 hours the hands overlap 11 times, in 24 hours they overlap 22 times.")
    ),
    Problem(
        name="Odd vs Even Sum",
        subject="Math",
        subtopic="Number Series",
        question=("Your friend Joey is fond of even numbers and you are fond of odd numbers. "
                 "One morning at 10:01 AM, Joey asks you a question: "
                 "Can you tell me the difference between the sum of odd numbers and the sum of even numbers up to 101 (included), if we start from 1?"),
        hint="",
        solution="51",
        explanation=("Look at the series: 1, 2, 3, 4, 5, ..., 101. "
                     "We need to compute (1 + 3 + ... + 101) - (2 + 4 + ... + 100). "
                     "Pairing them up: (1 - 2) + (3 - 4) + ... + (99 - 100) results in -50 (as there are 50 pairs). "
                     "The last number, 101, has no pair, so the final sum is 101 - 50 = 51.")
    ),

   Problem(
    name="Counting the Threes",
    subject="Math",
    subtopic="Counting",
    question=("How many positive integers are there from 1 to 400 (inclusive) that contain the digit 3?"),
    hint="Consider breaking the range into hundreds, tens, and units and count occurrences of the digit 3.",
    solution="157",
    explanation=("There are 100 numbers from 300 to 399 that have the number 3. "
                 "In addition, there are 30 numbers that have a unit digit of 3 and 30 numbers that have a tens digit of 3. "
                 "However, we overcounted three numbers, 33, 133, and 233. "
                 "So, our final count is 100 + 30 + 30 - 3 = 157 numbers that contain the digit 3.")
),

Problem(
    name="A Desperate Average",
    subject="Math",
    subtopic="Basic Operations",
    question=("Anya likes to go to school very much. She likes playing with her friends. "
              "On her first test, she got 1 out of 7. If she doesn't maintain an average of 5 out of 7, "
              "she will be separated from her friends and will be put in a different section. "
              "But she doesn't want to be separated from her friends. "
              "How many more tests does she need to give to maintain the average marks if she gets full marks in all of them?"),
    hint="Use the formula for the average and solve for the required number of tests.",
    solution="2",
    explanation=("Anya got 1 on her first exam. Let us suppose that Anya got 7 in the next n exams. "
                 "So the total number of exams including the first one will be n+1. "
                 "So, taking the average, we get (1+7n)/(n+1). "
                 "Now, (1+7n)/(n+1)=5 or, 1+7n=5n+5 or, n=2. "
                 "So, the number of tests Anya needs to give is 2.")
),

Problem(
    name="Dice Game",
    subject="Logical Reasoning",
    subtopic="Logical Problems",
    question=("When a fair six-sided die is tossed on a table, the bottom face cannot be seen. "
              "What is the probability that the product of the faces of the die that can be seen is divisible by 6?"),
    hint="Consider when the invisible face is 6 versus when it is not.",
    solution="1",
    explanation=("Look, we have only 2 cases to consider. "
                 "If the invisible face is anything other than 6, the product must be divisible by 6 "
                 "(since we are multiplying 6 with some other numbers when finding the product). "
                 "Now in the second case, if the invisible face is 6, our product will be 1×2×3×4×5=120, "
                 "which is also divisible by 6. Hence, the probability will be 1.")
),

Problem(
    name="Two Palindromes",
    subject="Math",
    subtopic="Number Theory",
    question=("A palindrome is a number that is equal to itself when read backward. "
              "Example: 363, 4224, 21512, etc. "
              "x is a 4-digit palindrome number and x + 212 is a 5-digit palindrome number. "
              "What is the sum of the digits of x?"),
    hint="Analyze the possible values of x that, when increased by 212, still form a palindrome.",
    solution="34",
    explanation=("The maximum possible value of x is 9999, which gives a maximum value for x + 212 as 10211. "
                 "The minimum value of x + 212 is 10000 as it's a 5-digit number. "
                 "We get that the first two digits of x must be 10, hence the last two must be 01 (since it's a palindrome). "
                 "There are only 3 possible cases for x + 212: 10001, 10101, or 10201. "
                 "Only 10101 gives a valid palindrome value for x, which is 9889, and the sum of its digits is 34.")
)


]
